In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_APIKEY'
os.environ['PINECONE_API_KEY'] = 'YOUR_PINECONE_APIKEY'

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pinecone import Pinecone

In [ ]:
openAIEmbeddings = OpenAIEmbeddings(model="text-embedding-3-large")
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
index_name = "curryproductinfo"
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [ ]:
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
    
if index_name in existing_indexes:
    index = pc.Index(index_name)

In [ ]:
class CreatePlomptMessages:
    def __init__(self):
        """
        CreatePlomptMessages
        """
        pass
    
    def queryModification(self, query, model):
        prompt_template = ChatPromptTemplate.from_messages([
            ("system",
            """
            あたなはRAG検索を行うためのクエリ生成のプロフェッショナルです。
            - 特にRAG検索を行う際は、「XXXでは無い」、「XXXが含まれてない」、「XXX以外の」など、否定的な意味を修飾する単語をを削除して、「YYYな商品」、「YYYが含まれている商品」などのクエリに変換する必要があります。
            - また、RAG検索を行うにあたって、ユーザーのメッセージが長い場合は要約して、検索しやすいクエリを生成してください。
            - 出力する内容は、生成したクエリ文のみにしてください。
            """),
            ("user", "{query}")
        ])
        parser = StrOutputParser()
        chain = prompt_template | model | parser
        return chain.invoke({"query": query})

    def summarizeSearchResults(self, product_info, query, model):
        prompt_template = ChatPromptTemplate.from_messages([
            ("system",
            """
            あたなはRAGの検索結果から、お客様に商品を案内するプロフェッショナルな店舗の営業マンです。
            ユーザーの問い合わせ内容から、以下内容を要約して、ユーザーにわかりやすく商品をレコメンドしてください。
            Docs\n\n {product_info},
            """),
            ("user", "{query}")
        ])
        parser = StrOutputParser()
        chain = prompt_template | model | parser
        return chain.invoke({"product_info": product_info, "query": query})

In [ ]:
query = "ベジタリアンでは無いカレーを教えてください"

In [ ]:
cpm = CreatePlomptMessages()

In [ ]:
newQuery = cpm.queryModification(query, llm)
print(newQuery)

In [ ]:
embedded_query = openAIEmbeddings.embed_query(newQuery)

In [ ]:
results = index.query(vector=embedded_query, top_k=5, include_metadata=True)

In [ ]:
content = ''
for result in results.matches:
    content+=str(result.metadata['text']) + '\n\n\n'
   

In [ ]:
output = cpm.summarizeSearchResults(content, query, llm)

In [ ]:
print(output)